# Face and Mask

In [1]:
import cv2
import matplotlib.pyplot as plt
import helpers
import numpy as np
import tensorflow as tf
from retinaface import RetinaFace
from PIL import Image

## Facial Recognition
We use retinaface to detect facial features

In [2]:
# Initialize facial detection
detector = RetinaFace(quality = "normal")

model[normal quality] init ..
model success !


In [3]:
# The actual detection of faces
img = detector.read("test.jpg")
faces = detector.predict(img)

In [4]:
# Cropping the faces and saving them
img = Image.open("test.jpg")
cropImages = []
for face in faces:
    cropImages.append(img.crop((face['x1'], face['y1'], face['x2'], face['y2'
    ])))

In [6]:
# Display the faces and original image
#%matplotlib inline
#plt.imshow(np.asarray(img))
#plt.figure()
#for i in cropImages:
#    plt.figure()
#    plt.imshow(i)

## Mask Detection
we will now pass cropImages through the mask classifier